In [5]:
%cd '/home/danbi/userdata/DANBI/gugakwon/Yeominrak'
%load_ext autoreload
%autoreload 2

from pathlib import Path
import music21
from music21 import converter, stream, note as m21_note 
from typing import List, Set, Dict, Tuple
import torch
from collections import defaultdict, Counter
import copy
import numpy as np

from yeominrak_processing import AlignedScore, SamplingScore, pack_collate, ShiftedAlignedScore
from model import Seq2seq, Converter, AttentionSeq2seq, QkvAttnSeq2seq, get_emb_total_size
import random as random
from loss import nll_loss
from trainer import Trainer
from decode import MidiDecoder
from torch.nn.utils.rnn import pack_sequence, PackedSequence, pad_packed_sequence, pack_padded_sequence
import torch
from torch.utils.data import Dataset, DataLoader
from omegaconf import OmegaConf
from metric import make_dynamic_template, convert_note_to_sampling, get_similarity, get_correspondence

/home/danbi/userdata/DANBI/gugakwon/Yeominrak
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = OmegaConf.load('yamls/baseline.yaml')
config = get_emb_total_size(config)
device = 'cpu'
val_dataset = ShiftedAlignedScore(is_valid= True, slice_measure_num=4)


In [3]:
valid_loader = DataLoader(val_dataset, batch_size=4, collate_fn=pack_collate, shuffle=False)
model = QkvAttnSeq2seq(val_dataset.tokenizer, config.model).to(device)
state = torch.load('best_model.pt')
model.load_state_dict(state)

<All keys matched successfully>

In [11]:
for i in range(8):
  num = 0
  for measure in val_dataset.measure_features[i]:
    if measure == []:
      num+=1
  print(i, 160 -num, num)

0 160 0
1 160 0
2 156 4
3 155 5
4 59 101
5 98 62
6 159 1
7 160 0


In [19]:
val_dataset.measure_features[2][40]

[[2, 51.0, 3.0, 0.0, 'strong'],
 [2, 51.0, 3.0, 3.0, 'middle'],
 [2, 51.0, 2.0, 6.0, 'strong'],
 [2, 48.0, 1.0, 8.0, 'middle'],
 [2, 46.0, 1.0, 9.0, 'weak']]

In [13]:
dynamic_template = make_dynamic_template(offset_list=val_dataset.offset_list)
whole_part_sampled_notes = []
for part_idx in range(8):
  part_sampled_notes = [convert_note_to_sampling(measure, dynamic_template) for measure in val_dataset.measure_features[part_idx]]
  whole_part_sampled_notes.append(part_sampled_notes)
  
idx_sample = [(0,1), (2,3), (3,4), (4,5), (5,6), (6,7)]
for idx_1, idx_2 in idx_sample:
  print('similirity between', idx_1,'-',idx_2, 'is', round(get_similarity(whole_part_sampled_notes, idx_1, idx_2), 5))
  print('dymamic corresponse between', idx_1, '-', idx_2, 'is', round(get_correspondence(whole_part_sampled_notes, idx_1, idx_2), 5))
  
  

similirity between 0 - 1 is 0.80625
dymamic corresponse between 0 - 1 is 0.84375
similirity between 2 - 3 is 0.92968
dymamic corresponse between 2 - 3 is 0.91935
similirity between 3 - 4 is 0.91102
dymamic corresponse between 3 - 4 is 0.95339
similirity between 4 - 5 is 0.91596
dymamic corresponse between 4 - 5 is 0.92021
similirity between 5 - 6 is 0.60238
dymamic corresponse between 5 - 6 is 0.78061
similirity between 6 - 7 is 0.9021
dymamic corresponse between 6 - 7 is 0.91038


In [24]:
def get_correspondence(idx_1, idx_2):
  correspondence = []
  for i in range(160):
    first = whole_part_sampled_notes[idx_1][i]
    second = whole_part_sampled_notes[idx_2][i]
    # print(first)
    if idx_1 in [2,3,4,5,6,7] or idx_2 in [4,5]:
      if first[0]==['empty'] or second[0]==['empty']:
        # print(first)
        continue
    strong_beat_a = []   
    strong_beat_b = []
    for j, note in enumerate(first):
      # print(note)
      if note[3] in ['strong', 'middle']:
        strong_beat_a.append(note[1])
    for j, note in enumerate(second):
      if note[3] in ['strong', 'middle']:
        strong_beat_b.append(note[1])
    
    measure_correspondence = sum([strong_beat_a[idx]==strong_beat_b[idx] for idx in range(len(strong_beat_a))])/len(strong_beat_a)
    correspondence.append(measure_correspondence)
   
  return sum(correspondence)/len(correspondence)


0 - 1 0.84375
2 - 3 0.91935
3 - 4 0.95339
4 - 5 0.92021
5 - 6 0.78061
6 - 7 0.91038


In [6]:
def make_dynamic_template(offset_list,beat_sampling_num=6):
  whole_dynamic_template = []
  for part_idx in range(8):
      frame = beat_sampling_num
      if part_idx == 0:
          # 세종실록악보 : 2.5/1.5, 1.5/1/1.5
          dynamic_mapping = {0.0: 'strong', 2.5: 'strong', 1.5: 'middle', \
              0.5: 'weak', 1.0: 'weak', 2.0: 'weak', 3.0: 'weak',3.5: 'weak'}
          frame = frame * 2 #세종실록인 경우는 frame이 두 배!
      
      elif part_idx == 1:
          # 금합자보: 5/3, 3/2/3
          dynamic_mapping = {0.0: 'strong', 5.0: 'strong', 3.0: 'middle',\
              1.0: 'weak', 2.0: 'weak', 4.0: 'weak', 6.0: 'weak', 7.0: 'weak',}
      
      elif part_idx in [2, 3, 4, 5]:
          # 속악원보, 금보신증가령, 한금신보, 어은보: 6/4, 3/3/2/2 
          dynamic_mapping = {0.0: 'strong', 6.0: 'strong', 3.0: 'middle', 8.0: 'middle', \
              1.0: 'weak', 2.0: 'weak', 4.0: 'weak', 5.0: 'weak', 7.0: 'weak', 9.0: 'weak'}
      
      elif part_idx in [6, 7]:
          # 삼죽금보, 현행: 5/5, 3/2/2/3
          dynamic_mapping = {0.0: 'strong', 5.0: 'strong', 3.0: 'middle', 7.0: 'middle',\
              1.0: 'weak', 2.0: 'weak', 4.0: 'weak', 6.0: 'weak', 8.0: 'weak', 9.0: 'weak'}
      whole_measure_len = int(frame * offset_list[part_idx])
      dynamic_list = []
      for i in range(whole_measure_len):
          dynamic = dynamic_mapping.get(float(i) / frame, 'pad')
          dynamic_list.append(dynamic)
      whole_dynamic_template.append(dynamic_list)
  return whole_dynamic_template

def convert_note_to_sampling(measure, dynamic_templates, beat_sampling_num = 6):
  if len(measure) == 0:
    return [['empty'] for _ in range(60)]  # 어차피, 4,5만 비어있으니까 괜찮을 것 같음! 
    
  new_measure = []
  for note in measure:
    frame = int(beat_sampling_num * note[2])
    if note[0]==0:
      frame = frame * 2 # beat strength는 절대값?
    # [part_idx, pitch, onset]
    new_note = [[note[0], note[1], 1]] + [[note[0], note[1], 0]]*(frame-1)
    new_measure += new_note
      
  dynamic_template = dynamic_templates[new_measure[0][0]] #part_idx
  dynamic_template = dynamic_template * (len(new_measure)//len(dynamic_template))
  new_measure_with_dynamic = []
  # print(len(new_measure), len(dynamic_template))
  for i, frame in enumerate(new_measure):
    if len(new_measure) == len(dynamic_template):
      new_frame = frame+[dynamic_template[i]]
      new_measure_with_dynamic.append(new_frame)
    else:
      ValueError('len(new_measure) != len(dynamic_template)')
  new_measure_with_dynamic

  return new_measure_with_dynamic

def get_similarity(idx_1, idx_2):
  similarity = []
  for i in range(160):
    first = whole_part_sampled_notes[idx_1][i]
    second = whole_part_sampled_notes[idx_2][i]
    
    if idx_1 in [2,3,4,5,6,7] or idx_2 in [4,5]:
      if first[0]==['empty'] or second[0]==['empty']:
        # print(first)
        continue
    measure_similarity = sum([note_1[1]==note_2[1] for note_1, note_2 in zip(first, second)])/len(first)
    similarity.append(measure_similarity)
   
  return sum(similarity)/len(similarity)